In [1]:
# Bibliotecas para EDA
import pandas as pd
import pingouin as pg
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

# Bibliotecas para Machine Learning
from sklearn.model_selection import cross_validate, StratifiedKFold, cross_val_predict, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# Otimização de parâmetros
import optuna

# EDA

In [2]:
# Carregar dataset
df = pd.read_csv('datasets/segmento_clientes.csv')

In [3]:
# Visualizar os dados
df.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,segmento_de_cliente
0,Comércio,713109.95,12,Rio de Janeiro,6,1,Bronze
1,Comércio,790714.38,9,São Paulo,15,0,Bronze
2,Comércio,1197239.33,17,São Paulo,4,9,Silver
3,Indústria,449185.78,15,São Paulo,6,0,Starter
4,Agronegócio,1006373.16,15,São Paulo,15,8,Silver
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,Silver
6,Serviços,771179.95,13,Vitória,0,1,Starter
7,Serviços,707837.61,16,São Paulo,10,6,Silver
8,Comércio,888983.66,17,Belo Horizonte,10,1,Bronze
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,Bronze


In [4]:
# Estrutura dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
 6   segmento_de_cliente     500 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 27.5+ KB


In [6]:
# Valores possíveis das variáveis categóricas

df['atividade_economica'].unique()

array(['Comércio', 'Indústria', 'Agronegócio', 'Serviços'], dtype=object)

In [7]:
df['localizacao'].unique()

array(['Rio de Janeiro', 'São Paulo', 'Vitória', 'Belo Horizonte'],
      dtype=object)

In [8]:
df['segmento_de_cliente'].unique()

array(['Bronze', 'Silver', 'Starter', 'Gold'], dtype=object)

In [9]:
# Conferir inovacao, que poderia ser categórica, por ser ordinal
df['inovacao'].unique()

array([1, 0, 9, 8, 4, 6, 3, 7, 5, 2])

In [11]:
# Verificar a distribuição dos valores da variável target: segmento_de_cliente
contagem_target = df.value_counts('segmento_de_cliente')
contagem_target

segmento_de_cliente
Silver     260
Bronze     202
Starter     22
Gold        16
Name: count, dtype: int64

- Temos uma base desbalanceada, com uma concentração de clientes muito alta em Silver e Bronze

In [14]:
# Criar lista ordenada do target
lista_segmentos = ['Starter', 'Bronze', 'Silver', 'Gold']

# Plotar a distribuição do target
px.bar(contagem_target, color=contagem_target.index, category_orders={'segmento_de_cliente': lista_segmentos})

In [15]:
# Plotar a distribuição do target - percentual
percentual_target = contagem_target / len(df) * 100
percentual_target

segmento_de_cliente
Silver     52.0
Bronze     40.4
Starter     4.4
Gold        3.2
Name: count, dtype: float64

In [16]:
# Plotar a distribuição do target
px.bar(percentual_target, color=percentual_target.index, category_orders={'segmento_de_cliente': lista_segmentos})

- Mais de 92% da base estão em Bronze e Silver

In [17]:
# Plotar a distribuição da Localização
percentual_localizacao = df.value_counts('localizacao') / len(df) * 100
px.bar(percentual_localizacao, color=percentual_localizacao.index)

In [18]:
# Plotar a distribuição da Atividade Econômica
percentual_atividade_economica = df.value_counts('atividade_economica') / len(df) * 100
px.bar(percentual_atividade_economica, color=percentual_atividade_economica.index)

In [19]:
# Plotar a distribuição da Inovação
percentual_inovacao = df.value_counts('inovacao') / len(df) * 100
px.bar(percentual_inovacao, color=percentual_inovacao.index)

- Apesar do desbalanceamento com o segmento de clientes, não temos o mesmo problema nas demais variáveis categóricas